# 📑 Report Fase 1: Analisi Dati e Strategia di Preprocessing

## 1. Obiettivo della Fase
L'obiettivo dello Step 1 è stato trasformare i dati grezzi di simulazione (file CSV eterogenei e parzialmente corrotti) in un formato numerico unificato e ottimizzato (tensore `.npy`), pronto per alimentare i modelli di Deep Learning.

## 2. Anatomia dei Dati (Mappa Strutturale)
Tutti i file di input condividono la stessa struttura matriciale "appiattita" (flattened). Ogni riga rappresenta un istante temporale ($t$), mentre le colonne rappresentano le coordinate spaziali dei **138 atomi** srotolate in sequenza.

**Mappa delle Colonne (per singolo file):**

| Indice Colonna | Contenuto | Appartenenza | Note |
| :--- | :--- | :--- | :--- |
| **0** | **Timestamp ($t$)** | - | *Variabile indipendente (da rimuovere)* |
| **1, 2, 3** | $x, y, z$ | **Atomo 1** | Prima terna |
| **4, 5, 6** | $x, y, z$ | **Atomo 2** | Seconda terna |
| ... | ... | ... | ... |
| **412, 413, 414** | $x, y, z$ | **Atomo 138** | Ultima terna |

* **Totale Colonne Utili:** $138 \text{ atomi} \times 3 \text{ coordinate} = \mathbf{414}$ feature.
* **Totale Righe:** $50.001$ step temporali.

---

## 3. Diagnostica dei Dataset (Risultati EDA)
L'Analisi Esplorativa dei Dati (EDA) ha evidenziato tre criticità fondamentali che sono state risolte durante il caricamento:

| Dataset | Contenuto ($\vec{x}, \vec{v}, \vec{F}$) | Range Numerico (Approx) | Stato del File (Parsing) | Azione Correttiva |
| :--- | :--- | :--- | :--- | :--- |
| **Coordinates** | Posizione spaziale | $\approx [20, 60]$ | ✅ **Pulito** (CSV Standard) | Nessuna riparazione necessaria. |
| **Velocity** | Velocità istantanea | $\approx [-2, +2]$ | ⚠️ **Sporco** (Separatori errati) | Parsing robusto (Split su spazi). |
| **Forces** | Accelerazione/Spinta | $\approx [-6700, +6400]$ | ⚠️ **Sporco** (Tabulazioni miste) | Parsing robusto (Split su tab). |

### 🚨 Il Problema del "Scale Mismatch"
È emerso un enorme squilibrio negli ordini di grandezza tra le variabili fisiche:
* Le **Forze** (ordine $10^3$) sono circa $1000\times$ più grandi delle **Velocità** (ordine $10^0$).
* **Implicazione:** Senza normalizzazione, la Rete Neurale ignorerà le velocità durante il calcolo del gradiente, causando errori sistematici nel training.

---

## 4. Strategia di Merge (Il "Super Dataset")
Per addestrare il modello a predire la dinamica completa, è stato creato un unico dataset concatenando le informazioni fisiche.

**Operazioni Eseguite:**
1.  **Drop Timestamp:** La colonna 0 è stata rimossa da tutti i file (l'informazione temporale è implicita nella sequenza delle righe).
2.  **Concatenazione Orizzontale:** I tre tensori ($X, V, F$) sono stati uniti fianco a fianco (`axis=1`).

**Dimensione Finale del Dataset:**
Il file salvato `dataset_completo.npy` ha la seguente shape:
$$\text{Shape} = (50.001 \text{ righe}, \mathbf{1242} \text{ colonne})$$

Dove il numero di feature $1242$ deriva da: $414 \text{ (Coords)} + 414 \text{ (Vels)} + 414 \text{ (Forces)}$.

---